# Instalação
#INSTALAÇÃO DAS LIBS
- !pip install yfinance==0.2.41
- !pip install crewai==0.28.8
- !pip install 'crewai[tools]'
- !pip install langchain==0.1.20
- !pip install langchain-openai==0.1.7
- !pip install langchain-community==0.0.38
- !pip install duckduckgo-search==5.3.0

In [105]:
import json
import os

from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import display, Markdown

In [83]:
# CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock


yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func=lambda ticket: fetch_stock_price(ticket)
)

In [84]:
print(fetch_stock_price('AAPL'))

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043777   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546387   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         

In [85]:
import os

# Importando OpenAI LLM - GPT
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o")

In [86]:
stockPriceAnalyst = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analyses trends",
    backstory="""You're highly experienced in analyzing the price of an specific stock
    and make predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
    allow_delegation=False
)

In [87]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output="""" Specify the current trend stock price - up, down or sideways. 
    eg. stock= 'APPL, price UP'
""",
    agent=stockPriceAnalyst
)

In [88]:
# IMPORTANT A TOOL DE SEARCH
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [89]:
newsAnalyst = Agent(
    role="Stock News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with
    the news context. For each request stock asset, specify a numbet between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory="""You're highly experienced in analyzing the market trends and news and have tracked assest for more then 10 years.
    You're also master level analyts in the tradicional markets and have deep understanding of human psychology.
    You understand news, theirs tittles and information, but you look at those with a health dose of skepticism. 
    You consider also the source of the news articles. 
    """,
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
    allow_delegation=False
)

In [90]:
get_news = Task(
    description=f"""Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually. 
    The current date is {datetime.now()}.
    Compose the results into a helpfull report""",
    expected_output=""""A summary of the overall market and one sentence summary for each request asset. 
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
""",
    agent=newsAnalyst
)

In [91]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyts Writer",
    goal=""""Analyze the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend. """,
    backstory="""You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences. 
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multiple opinions when analyzing anything.
""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True
)

In [92]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
""",
    expected_output= """"An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:
    - 3 bullets executive summary 
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fead/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
    write in pt-br
""",
    agent = stockAnalystWrite,
    context = [getStockPrice, get_news]
)

In [93]:
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = 2,
    process= Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)

2024-08-25 20:24:30,638 - 140700230648256 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [94]:
results = crew.kickoff(inputs={"ticket": "AAPL"})


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
To analyze the stock AAPL price history and create a trend analysis, we need to gather data on the stock's price history, any relevant news that might have influenced the stock price, and then interpret this information to determine if the trend is up, down, or sideways. 

First, I'll delegate the task of analyzing the stock price history to the Senior Stock Price Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior stock price Analyst", "task": "Analyze the price history of AAPL", "context": "We need a detailed analysis of the historical price data for AAPL to understand its trend. Please provide an overview of the price movements over the past year and any noticeable patterns or trends. This information is crucial for determining whether the current trend is up, dow

[*********************100%%**********************]  1 of 1 completed

Thought: To analyze the price history of AAPL, I need to fetch the stock prices for AAPL from the last year using the Yahoo Finance Tool.

Action: Yahoo Finance Tool
Action Input: {"ticker": "AAPL"}
 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043777   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546387   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004

Thought: I now have the historical price data for AAPL over the past year. I need to analyze this data to identify trends and patterns.

Final Answer: 

# AAPL Stock Price Analysis (August 2023 - August 2024)

## Overview:
The data retrieved contains the Open, High, Low, Close, Adjusted Close prices, and Volume for AAPL stock for each trading day from August 8, 2023, to August 7, 2024. 

## Key Observations:
- The highest closing price in the dataset is observed around early August 2024, at approximately $225.60.
- The lowest closing price in the dataset during this period is around early August 2023, with prices around $177.58.
- The stock experienced significant volatility, with notable price fluctuations, especially towards the end of the period (July - August 2024).

## Detailed Analysis:
1. **August 2023:** The stock price started around $179.80 and fluctuated within the $176 - $180 range. This period shows relatively minor fluctuations.
   
2. **September 2023 to December 2023:**

[*********************100%%**********************]  1 of 1 completed

Thought: I need to fetch the stock prices for AAPL from the last year using the Yahoo Finance Tool to analyze its trend. Additionally, I need to include BTC in the analysis.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}
 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643387   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043777   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546387   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996


[*********************100%%**********************]  1 of 1 completed

Thought: I need to fetch the stock prices for BTC from the last year using the Yahoo Finance Tool to include it in my analysis.

Action: Yahoo Finance Tool
Action Input: {"ticket": "BTC-USD"} 

                    Open          High           Low         Close  \
Date                                                                 
2023-08-08  29180.019531  30176.796875  29113.814453  29765.492188   
2023-08-09  29766.695312  30093.435547  29376.800781  29561.494141   
2023-08-10  29563.972656  29688.564453  29354.447266  29429.591797   
2023-08-11  29424.902344  29517.773438  29253.517578  29397.714844   
2023-08-12  29399.787109  29465.113281  29357.587891  29415.964844   
...                  ...           ...           ...           ...   
2024-08-03  61414.808594  62148.371094  59836.527344  60680.093750   
2024-08-04  60676.093750  61062.988281  57210.804688  58116.976562   
2024-08-05  58110.296875  58268.828125  49121.238281  53991.457031   
2024-08-06  53991.347656  57059.9179

Thought: I now have the stock prices for both AAPL and BTC from the last year. I will analyze their trends to provide a comprehensive analysis.

### AAPL Stock Analysis

Over the past year, AAPL has shown a general upward trend. Here are some key observations:
1. **Initial Price (2023-08-08)**: $179.80
2. **Final Price (2024-08-07)**: $209.82
3. **Highest Price**: $225.60 (2024-08-02)
4. **Lowest Price**: $176.55 (2023-08-11)

The stock had consistent growth, with some notable peaks and troughs. The highest peak was in early August 2024, suggesting strong performance likely due to positive financial results or market sentiment.

### BTC Stock Analysis

BTC-USD has also shown an upward trend but with more volatility:
1. **Initial Price (2023-08-08)**: $29,765.49
2. **Final Price (2024-08-07)**: $55,027.46
3. **Highest Price**: $62,148.37 (2024-08-03)
4. **Lowest Price**: $29,113.81 (2023-08-08)

BTC experienced significant growth, especially in the first half of 2024. The volatility is 

In [96]:
list(results.keys())

['final_output', 'tasks_outputs']

In [101]:
len(results['tasks_outputs'])

3

In [107]:
Markdown(results['final_output'])

**Apple Inc. (AAPL) Market Newsletter**

- **Resumo Executivo:**
  - AAPL enfrenta uma tendência lateral a ligeiramente descendente devido a problemas de lançamento de produtos e questões na cadeia de suprimentos.
  - O sentimento do mercado é neutro, com um índice de medo/ganância de 60, indicando otimismo moderado com cautela.
  - O relatório de ganhos do Q4 e a resolução dos problemas na cadeia de suprimentos serão cruciais para o desempenho futuro das ações.

**Introdução:**
Atualmente, os mercados financeiros estão navegando por um cenário complexo caracterizado por sinais mistos em diferentes classes de ativos. Enquanto ações tradicionais como AAPL estão passando por períodos de consolidação e incerteza, outras classes de ativos, como criptomoedas, estão em fases de alta. Essa dualidade destaca a importância de estratégias de investimento diversificadas e um olhar atento tanto para fatores macroeconômicos quanto para desenvolvimentos específicos de ativos.

**Análise Principal:**
A Apple Inc. está enfrentando uma tendência lateral a ligeiramente descendente devido a relatórios mistos sobre lançamentos de produtos e problemas contínuos na cadeia de suprimentos. O sentimento em torno da AAPL é neutro, com um índice de sentimento de mercado de 50, indicando uma abordagem de espera e observação entre os investidores. Reações mistas ao lançamento do iPhone 15 e as contínuas interrupções na cadeia de suprimentos são influências principais que dirigem essa tendência. A antecipação em torno do próximo relatório de ganhos do Q4 adiciona outra camada de incerteza, tornando crucial para os investidores ficarem informados sobre possíveis resoluções desses problemas de cadeia de suprimentos e quaisquer novos desenvolvimentos no desempenho dos produtos.

**Resumo:**
Olhando para o futuro, a perspectiva de curto prazo para a AAPL sugere uma consolidação contínua com riscos potenciais de baixa se os problemas na cadeia de suprimentos persistirem ou se os próximos lançamentos de produtos não atenderem às expectativas do mercado. Os investidores devem monitorar de perto os relatórios de ganhos e quaisquer atualizações sobre as resoluções na cadeia de suprimentos para avaliar o desempenho futuro. Apesar da atual volatilidade, a análise de sentimento coloca a AAPL em um índice de medo/ganância de 60, indicando otimismo moderado com cautela. Ao sintetizar esses insights, os investidores podem navegar melhor a dinâmica atual do mercado e tomar decisões informadas, entendendo que os próximos meses serão cruciais para determinar a direção futura das ações.